In [1]:
from sklearn import preprocessing


#--------------------------------------------------
#todo: 过滤基因
#* 至少在一个时期中80%的样本中FPKM>=5
#--------------------------------------------------
import pandas as pd
import numpy as np 
import sys 
stageArray=['0DPA','4DPA','8DPA','12DPA','16DPA','20DPA']
readCountData=[]
AllsampleName=pd.read_csv("./All_sampleName.txt",header=None,index_col=None,sep="\t")
#* get read count of each gene at times scheme
for stage in stageArray:
    read_count = pd.read_csv(
    "../Gene_align/read_count/"+stage+"/geneFPKM.txt", header=0, index_col=0, sep="\t")
    read_count.columns=AllsampleName.loc[AllsampleName[0].isin(read_count.columns)].index
    readCountData.append(read_count)

In [25]:
geneIDFile='/data/cotton/zhenpingliu/LZP_fiberFullPopulationRNAseq/03express_gene/eQTLgenes_Allstages_340samples/fastQTL_peer_20/Gene_information/All_gene_TSS.bed'
geneList=pd.read_csv(geneIDFile,header=0,index_col=None,sep="\t")
filterGene=[]
for geneid in geneList['geneId']:
    fullFillSample=[]
    for readData in readCountData:
        tmpData=readData.loc[geneid]
        fullFillSample.append(
            (len([i for i in tmpData if i>=5]),readData.shape[1])
        ) 
    if len([j for i,j in fullFillSample if i/j>=0.8])>=1:
        filterGene.append(geneid)
filterGene=pd.DataFrame(filterGene)
filterGene.to_csv("filterGene.txt",header=None,index=False,sep="\t")        

In [2]:
from math import log2
def get_lm_matrix(readCountArray, geneId):
    ''' format data to run lme4 regression model.
        + readCountArray: each time point read count.
    '''
    # * scale time to mean zeri variance 1.
    # ?  six time point
    time_array = preprocessing.scale(range(1, 7, 1))
    # *  expression  sampleId, scaled time
    readMatrix = np.empty((0, 3))
    for index, readData in enumerate(readCountArray):
        #* log2(FPKM+1) transform the FPKM
        timePoint=time_array[index]
        for sampleIndex,FPKM in readData.loc[geneId].iteritems():
            tmpData=[log2(FPKM+1), sampleIndex,timePoint]
            readMatrix = np.vstack([readMatrix,tmpData])
    readMatrix = pd.DataFrame(
        readMatrix, columns=['FPKM', 'sampleId', 'time'])
    return readMatrix


from rpy2.robjects import pandas2ri
import rpy2.robjects as robjects
pandas2ri.activate()
robjects.r('''
        library(lme4)  
        library(aod)
        ''')

RunAnalyseMLMlogit = robjects.r(
    '''
       f <- function(readMatrix){
        #*---------------------------
        #* 1. log2(FPKM+1) 
        #* 2. sampleId 
        #* 3. time
        #*---------------------------
        # readMatrix$refAllele=as.factor(readMatrix$sampleId)
        # readMatrix$timepoint =as.factor(readMatrix$time)
        #*------------------------------
        #* Variance decomposition 
        #*------------------------------
        model=lmer('FPKM ~ 1 + (1 | time) +(1|sampleId)', data = readMatrix)
        
        timeVariant=as.data.frame(summary(model)$varcor)[1,4]
        sampleVariant=as.data.frame(summary(model)$varcor)[2,4]
        residualVariant=as.data.frame(summary(model)$varcor)[3,4]
        result <- c(timeVariant,sampleVariant,residualVariant)
        return (result)
       } 
    '''
)

def bootstrap(lm_matrix,sampleTimes=100):
    VariantResult=[]
    for item in range(0,sampleTimes):
        tmeData=lm_matrix.sample(frac=1,replace=True)
        timeVariant,sampleVariant,residualVariant=RunAnalyseMLMlogit(tmeData)
        VariantResult.append(
            (
                timeVariant/(timeVariant+sampleVariant+residualVariant),
                sampleVariant/(timeVariant+sampleVariant+residualVariant)
            )
        )
    #! 95% confidence interval
    #* 2.5 Quantile and 97.5 Quantile
    VariantResult=pd.DataFrame(VariantResult,columns=['tiems','samples'])
    confidenceInterVal=[]
    for index,Variant in VariantResult.iteritems():
        meanData=np.mean(Variant)
        lowData=np.percentile(Variant,2.5)
        highData=np.percentile(Variant,97.5)
        confidenceInterVal.append((lowData,meanData,highData))
    #----------------------------------
    #* confidence InterVal Array
    #[
    # [ low, mean,high]
    # ]
    #----------------------------------
    return confidenceInterVal

R[write to console]: Loading required package: Matrix



In [3]:
dataMatrix=get_lm_matrix(readCountData,'Ghir_A05G006800')
timeInterVal,sampleVal=bootstrap(dataMatrix)

((0.1635121872318504, 0.19781410263624769, 0.2379266863732302),
 (0.08555661079019339, 0.10728577588692356, 0.1341924516029755))

In [9]:
get_lm_matrix(readCountData,'Ghir_A05G006800')

,FPKM,sampleId,time
0,3.862461,0.0,-1.46385
1,3.497804,1.0,-1.46385
2,3.725381,2.0,-1.46385
3,3.744559,3.0,-1.46385
4,3.773237,4.0,-1.46385
...,...,...,...
2210,3.809106,371.0,1.46385
2211,3.541664,372.0,1.46385
2212,3.504786,373.0,1.46385
2213,3.816702,374.0,1.46385


In [4]:
dataMatrix.to_csv("test_data.txt",header=True,index=False,sep="\t")

In [5]:
RunAnalyseMLMlogit(robjects.pandas2ri.DataFrame(dataMatrix))

array([0.0045037 , 0.08627789, 0.06126121])